# 필요 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

import random
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings(action = 'ignore')

# 데이터 불러오기

In [ ]:
data = pd.read_csv('../data/recomm/user_item_interactions.csv')
data = data.rename(columns={'TRAVELER_ID': 'user_id', 'POI_ID': 'item_id'})
data.head()

### 긍정 상호 작용 데이터 설정 (별점 3점이상)

In [ ]:
data = data[data['STARS'] >= 3]
len(data)

In [ ]:
data.groupby(['STARS'])['STARS'].count()

# 데이터 전처리

## Train & Test split

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data.values, test_size = 0.2, random_state = 42)
train = pd.DataFrame(train, columns = data.columns)
test = pd.DataFrame(test, columns = data.columns)

In [ ]:
print('Train size : ', len(train))
print('Test size : ', len(test))

## Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

user_label_encoder = LabelEncoder()
item_label_encoder = LabelEncoder()

train['user_id_idx'] = user_label_encoder.fit_transform(train['user_id'].values)
train['item_id_idx'] = item_label_encoder.fit_transform(train['item_id'].values)

In [ ]:
train_user_ids = train['user_id'].unique()
train_item_ids = train['item_id'].unique()
print("훈련 데이터셋에 있는 유저 수:", len(train_user_ids), "훈련 데이터셋에 있는 아이템 수:", len(train_item_ids))

In [ ]:
test = test[(test['user_id'].isin(train_user_ids)) & (test['item_id'].isin(train_item_ids))]
test['user_id_idx'] = user_label_encoder.transform(test['user_id'].values)
test['item_id_idx'] = item_label_encoder.transform(test['item_id'].values)
print("테스트 데이터셋 크기:", len(test))

# 모델 구축

In [ ]:
# dok_mtrx를 COO 형식으로 변환하여 희소 텐서로 변환하는 함수
def convert_to_sparse_tensor(dok_mtrx):
    # dok_mtrx를 COO 형식으로 변환
    dok_mtrx_coo = dok_mtrx.tocoo().astype(np.float32)

    # COO 형식으로 변환된 행렬에서 값과 인덱스 추출
    values = dok_mtrx_coo.data
    indices = np.vstack((dok_mtrx_coo.row, dok_mtrx_coo.col))

    # 인덱스를 torch.LongTensor로 변환하고 값은 torch.FloatTensor로 변환
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)

    # 행렬의 모양 추출
    shape = dok_mtrx_coo.shape

    # 변환된 값과 인덱스를 사용하여 희소 텐서 생성
    dok_mtrx_sparse_tensor = torch.sparse.FloatTensor(i, v, torch.Size(shape))

    # 생성된 희소 텐서 반환
    return dok_mtrx_sparse_tensor

In [ ]:
import scipy.sparse as sparse

# 사용자 및 항목 간의 관련성 점수를 계산하고 평가 메트릭을 반환하는 함수
def get_metrics(W_u, W_i, n_users, n_items, train_data, test_data, K):
    # 테스트에 사용되는 사용자 ID 추출
    test_user_ids = torch.LongTensor(test_data['user_id_idx'].unique())
    
    # 사용자 및 항목 간의 관련성 점수 계산
    relevance_score = torch.matmul(W_u, torch.transpose(W_i, 0, 1))
    
    # 훈련 데이터로 희소 행렬 생성
    R = sparse.dok_matrix((n_users, n_items), dtype=np.float32)
    R[train_data['user_id_idx'], train_data['item_id_idx']] = 1.0
    
    # 희소 행렬을 PyTorch 희소 텐서로 변환
    R_tensor = convert_to_sparse_tensor(R)
    
    # 희소 텐서를 밀집 텐서로 변환하고 무한대를 0으로 대체
    R_tensor_dense = R_tensor.to_dense()
    R_tensor_dense = R_tensor_dense * (-np.inf)
    R_tensor_dense = torch.nan_to_num(R_tensor_dense, nan=0.0)
    
    # 관련성 점수에 훈련 데이터를 추가하여 아이템이 이미 상호 작용한 경우 관련성 점수를 음의 무한대로 설정
    relevance_score = relevance_score + R_tensor_dense
    
    # 상위 K개의 인덱스 추출
    topk_idx = torch.topk(relevance_score, K).indices
    
    # 상위 K개의 인덱스를 데이터프레임으로 변환
    topk_idx_df = pd.DataFrame(topk_idx.numpy(), columns=['top_idx' + str(x + 1) for x in range(K)])
    topk_idx_df['user_id'] = topk_idx_df.index
    topk_idx_df['topk_item'] = topk_idx_df[['top_idx' + str(x + 1) for x in range(K)]].values.tolist()
    topk_idx_df = topk_idx_df[['user_id', 'topk_item']]
    
    # 테스트 데이터의 아이템 목록 추출
    test_items = test_data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    
    # 테스트 데이터와 상위 K개의 아이템 목록을 결합
    metrics_df = pd.merge(test_items, topk_idx_df, how='left', left_on='user_id_idx', right_on='user_id')
    
    # 테스트 데이터와 상위 K개의 아이템 목록에서 상호 작용하는 아이템 수 계산
    metrics_df['interact_items'] = [list(set(a).intersection(b)) for a, b in zip(metrics_df.item_id_idx, metrics_df.topk_item)]
    
    # 리콜 계산
    metrics_df['recall'] = metrics_df.apply(lambda x: len(x['interact_items']) / len(x['item_id_idx']), axis=1)
    
    # DCG 및 nDCG 계산을 위한 함수 정의
    def get_dcg_idcg(item_id_idx, hit_list):
        dcg = sum([hit * np.reciprocal(np.log1p(idx + 1)) for idx, hit in enumerate(hit_list)])
        idcg = sum([np.reciprocal(np.log1p(idx + 1)) for idx in range(min(len(item_id_idx), len(hit_list)))])
        return dcg / idcg
    
    # 히트 목록 및 nDCG 계산
    metrics_df['hit_list'] = metrics_df.apply(lambda x: [1 if i in set(x['item_id_idx']) else 0 for i in x['topk_item']], axis=1)
    metrics_df['ndcg'] = metrics_df.apply(lambda x: get_dcg_idcg(x['item_id_idx'], x['hit_list']), axis=1)
    
    # 평균 리콜 및 nDCG 반환
    return metrics_df['recall'].mean(), metrics_df['ndcg'].mean()

In [ ]:
class LightGCN(nn.Module):
    def __init__(self, data, n_users, n_items, n_layers, latent_dim):
        super(LightGCN, self).__init__()
        self.data = data
        self.n_users = n_users
        self.n_items = n_items
        self.n_layers = n_layers
        self.latent_dim = latent_dim
        self.init_embedding()
        self.norm_adj_mat_sparse_tensor = self.get_norm_adj()
    
    # 초기 임베딩을 설정하는 메서드
    def init_embedding(self):
        self.E0 = nn.Embedding(self.n_users + self.n_items, self.latent_dim)
        nn.init.xavier_uniform_(self.E0.weight)
        self.E0.weight = nn.Parameter(self.E0.weight)
    
    # 정규화된 인접 행렬을 생성하는 메서드
    def get_norm_adj(self):
        R = sparse.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        R[self.data['user_id_idx'], self.data['item_id_idx']] = 1.0
        adj_mat = sparse.dok_matrix((self.n_users + self.n_items, self.n_users + self.n_items), dtype=np.float32)
        adj_mat[:self.n_users, self.n_users:] = R
        adj_mat[self.n_users:, :self.n_users] = R.T
        rowsum = np.array(adj_mat.sum(1))
        d_inv = np.power(rowsum + 1e-9, -0.5).flatten()
        d_mat_inv = sparse.diags(d_inv)
        
        # D^(-1/2) _A_ D^(-1/2) 계산
        norm_adj_mat = d_mat_inv.dot(adj_mat).dot(d_mat_inv)
        norm_adj_mat_coo = norm_adj_mat.tocoo().astype(np.float32)
        values = norm_adj_mat_coo.data
        indices = np.vstack((norm_adj_mat_coo.row, norm_adj_mat_coo.col))
        i = torch.LongTensor(indices)
        v = torch.FloatTensor(values)
        return torch.sparse.FloatTensor(i, v, torch.Size(norm_adj_mat_coo.shape))
    
    # 레이어를 통과시키는 메서드
    def propagate_through_layers(self):
        all_layers_embedding = [self.E0.weight]
        E_lyr = self.E0.weight
        for layer in range(self.n_layers):
            E_lyr = torch.sparse.mm(self.norm_adj_mat_sparse_tensor, E_lyr)
            all_layers_embedding.append(E_lyr)
        all_layers_embedding = torch.stack(all_layers_embedding)
        mean_layer_embedding = torch.mean(all_layers_embedding, axis=0)
        return torch.split(mean_layer_embedding, [self.n_users, self.n_items]) + torch.split(self.E0.weight, [self.n_users, self.n_items])
    
    # 모델의 순전파를 정의하는 메서드
    def forward(self, users, pos_items, neg_items):
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = self.propagate_through_layers()
        return (final_user_emb[users], final_item_emb[pos_items], final_item_emb[neg_items], initial_user_emb[users], initial_item_emb[pos_items], initial_item_emb[neg_items])

In [ ]:
def bpr_loss(users, users_emb, pos_emb, neg_emb, user_emb_0, pos_emb_0, neg_emb_0):
    # positive 및 negative 샘플에 대한 점수 계산
    pos_scores = torch.sum(torch.mul(users_emb, pos_emb), dim=1)
    neg_scores = torch.sum(torch.mul(users_emb, neg_emb), dim=1)
    
    # Bayesian Personalized Ranking (BPR) 손실 계산
    loss = -torch.mean(torch.log(torch.sigmoid(pos_scores - neg_scores)))
    
    # 정규화 항 추가
    if config.reg > 0:
        l2_norm = (user_emb_0.norm().pow(2) + pos_emb_0.norm().pow(2) + neg_emb_0.norm().pow(2)) / float(len(users))
        reg_loss = config.reg * l2_norm
        loss += reg_loss
    
    return loss

# 모델학습

In [ ]:
config = {
    'latent_dim' : 64,
    'lr' : 0.001,
    'batch_size' : 1024,
    'top_k' : 10,
    'n_layers' : 3,
    'reg' : 1e-4,
    'epochs' : 300
}

In [ ]:
n_users = train['user_id_idx'].nunique()
n_items = train['item_id_idx'].nunique()

In [ ]:
from box import Box
config = Box(config)

In [ ]:
lightGCN = LightGCN(train, n_users, n_items, config.n_layers, config.latent_dim)

In [ ]:
def data_loader(data, batch_size, n_users, n_items):
    # 사용자별 상호 작용한 항목 목록 추출
    interacted_items_df = data.groupby('user_id_idx')['item_id_idx'].apply(list).reset_index()
    
    # 부정적 샘플링 함수 정의
    def sample_neg(x):
        while True:
            neg_id = random.randint(0, n_items - 1)
            if neg_id not in x:
                return neg_id
    
    # 랜덤하게 사용자 선택
    indices = [x for x in range(n_users)]
    if n_users < batch_size:
        users = [random.choice(indices) for _ in range(batch_size)]
    else:
        users = random.sample(indices, batch_size)
    users.sort()
    
    # 선택된 사용자와 상호 작용한 항목을 결합하여 데이터프레임 생성
    users_df = pd.DataFrame(users, columns=['users'])
    interacted_items_df = pd.merge(interacted_items_df, users_df, how='right', left_on='user_id_idx', right_on='users')
    
    # 양성 샘플 및 부정적 샘플 선택
    pos_items = interacted_items_df['item_id_idx'].apply(lambda x: random.choice(x)).values
    neg_items = interacted_items_df['item_id_idx'].apply(lambda x: sample_neg(x)).values
    
    return list(users), list(pos_items), list(neg_items)

## 하이퍼 파라미터 튜닝

In [ ]:
import optuna
import math

def objective(trial):
    # 하이퍼파라미터 설정
    config = {
        'latent_dim': trial.suggest_categorical('latent_dim', [32, 64, 128]),
        'lr': trial.suggest_loguniform('lr', 1e-4, 1e-2),
        'batch_size': trial.suggest_categorical('batch_size', [256, 512, 1024]),
        'num_layers': trial.suggest_int('num_layers', 1, 5),
        'reg': trial.suggest_loguniform('reg', 1e-6, 1e-3),
        'epochs': 50,  # 튜닝을 위해 에포크 수를 줄임
        'top_k': 10
    }
    
    # n_batch 계산
    n_batch = math.ceil(len(train) / config['batch_size'])  # train_dataset의 총 개수에 따라 조정 필요
    
    # 모델 초기화
    model = LightGCN(train, n_users, n_items, config['num_layers'], config['latent_dim'])
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['reg'])
    
    # 학습 루프
    for epoch in range(config['epochs']):
        model.train()
        loss_epoch = []
        
        for batch_idx in range(n_batch):
            optimizer.zero_grad()
            users, pos_items, neg_items = data_loader(train, config['batch_size'], n_users, n_items)
            # 올바른 모델 호출을 위해 코드 수정
            users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0 = model.forward(users, pos_items, neg_items)
            loss = bpr_loss(users, users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0)
            loss.backward()
            optimizer.step()
            loss_epoch.append(loss.item())
        
        model.eval()
        with torch.no_grad():
            final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = model.propagate_through_layers()
            test_recall, test_ndcg = get_metrics(final_user_emb, final_item_emb, n_users, n_items, train, test, config['top_k'])
        
        print(f"[에폭: {epoch+1}/{config['epochs']}, 손실: {np.mean(loss_epoch):.4f}, 리콜@{config['top_k']}: {test_recall:.4f}, NDCG@{config['top_k']}: {test_ndcg:.4f}]")
    
    return test_recall

# 하이퍼파라미터 튜닝 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)

In [ ]:
"""
optuna 로 최적 모델을 탐색했으나 커널이슈로 기록이 사라짐
이슈 발생전 기록해놓은 최적 파라미터값은 아래와 같음
"""
# 최적 파라미터로 50 epch를 학습한 모델 저장
# Trial 15 finished with value: 0.2492541342615162 and parameters: {'latent_dim': 64, 'lr': 0.009631732140395756, 
#'batch_size': 256, 'num_layers': 4, 'reg': 1.0794690314138785e-06}. Best is trial 15 with value: 0.2492541342615162.

## 베스트 하이퍼파라미터 학습

In [ ]:
# 최적 하이퍼파라미터 설정
#best_params = study.best_params


# 학습 이력을 바탕으로 확인한 최적의 하이퍼파라미터 설정
best_params = {
    'latent_dim': 64,  # 최적의 latent dimension 크기
    'lr': 0.009631732140395756,  # 최적의 learning rate
    'batch_size': 256,  # 최적의 batch size
    'num_layers': 4,  # 최적의 layer 수
    'reg': 1.0794690314138785e-06,  # 최적의 regularization strength
    'epochs': 200  # 최종 학습을 위해 조정한 에포크 수
}


best_params['epochs'] = 200  # 최종 학습을 위해 에포크 수 조정

# 최적 하이퍼파라미터로 모델 학습
lightGCN = LightGCN(train, n_users, n_items, best_params['num_layers'], best_params['latent_dim'])
optimizer = torch.optim.Adam(lightGCN.parameters(), lr=best_params['lr'], weight_decay=best_params['reg'])

In [ ]:
import math

top_k = 10  # top_k 값을 직접 설정

loss_list_epoch = []
recall_list = []
ndcg_list = []
n_batch = math.ceil(len(train) / best_params['batch_size'])  # train_dataset의 총 개수에 따라 조정 필요

for epoch in range(best_params['epochs']):
    lightGCN.train()
    final_loss_list = []
    
    for batch_idx in range(n_batch):
        optimizer.zero_grad()
        users, pos_items, neg_items = data_loader(train, best_params['batch_size'], n_users, n_items)
        users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0 = lightGCN.forward(users, pos_items, neg_items)
        final_loss = bpr_loss(users, users_emb, pos_emb, neg_emb, users_emb_0, pos_emb_0, neg_emb_0)
        final_loss.backward()
        optimizer.step()
        final_loss_list.append(final_loss.item())
    
    lightGCN.eval()
    with torch.no_grad():
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
        test_recall, test_ndcg = get_metrics(final_user_emb, final_item_emb, n_users, n_items, train, test, top_k)  # best_params['top_k'] 대신 직접 설정한 top_k 사용
    
    loss_list_epoch.append(np.mean(final_loss_list))
    recall_list.append(test_recall)
    ndcg_list.append(test_ndcg)
    
    print(f"[에폭: {epoch+1}/{best_params['epochs']}, 손실: {np.mean(final_loss_list):.4f}, 리콜@{top_k}: {test_recall:.4f}, NDCG@{top_k}: {test_ndcg:.4f}]")

In [ ]:
torch.save(lightGCN.state_dict(), '../model/LightGCN_best_model_weights.pth')

# 학습 결과 평가

In [ ]:
import matplotlib.pyplot as plt

# 학습 결과 시각화
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(loss_list_epoch)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 3, 2)
plt.plot(recall_list)
plt.title('Test Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')

plt.subplot(1, 3, 3)
plt.plot(ndcg_list)
plt.title('Test NDCG')
plt.xlabel('Epoch')
plt.ylabel('NDCG')

plt.tight_layout()
plt.show()

# 추천여행지 추출

In [ ]:
travel_log = pd.read_csv('../data/travel_log/travel_log.csv', encoding = 'utf-8')
travel_log.head()

In [ ]:
best_params = {
    'latent_dim': 64,  # 최적의 latent dimension 크기
    'lr': 0.009631732140395756,  # 최적의 learning rate
    'batch_size': 256,  # 최적의 batch size
    'num_layers': 4,  # 최적의 layer 수
    'reg': 1.0794690314138785e-06,  # 최적의 regularization strength
    'epochs': 200  # 최종 학습을 위해 조정한 에포크 수
}

# 저장된 모델 가중치 불러오기
model_weights = torch.load('../model/LightGCN_best_model_weights.pth')

# 최적 하이퍼파라미터로 모델 학습
"""
위에 LightGCN 클래스 선언 필수
"""
lightGCN = LightGCN(train, n_users, n_items, best_params['num_layers'], best_params['latent_dim'])

# 불러온 가중치를 모델에 적용
lightGCN.load_state_dict(model_weights)

In [ ]:
import pandas as pd

# 사용자 ID 입력받기
user_id = input("사용자 ID를 입력하세요: ")
user_idx = user_label_encoder.transform([user_id])[0]

lightGCN.eval()
with torch.no_grad():
    # 사용자 임베딩 가져오기
    final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
    user_embedding = final_user_emb[user_idx]

    # 아이템별 유사도 점수 계산
    similarity_scores = torch.matmul(user_embedding, final_item_emb.transpose(0, 1))
    recommended_item_indices = torch.topk(similarity_scores, top_k).indices

# 추천된 아이템의 상세 정보를 포함한 데이터 프레임 생성
recommended_items = []

# 추천된 아이템들의 상세 정보 추출
for item_idx in recommended_item_indices:
    item_id = item_label_encoder.inverse_transform([item_idx.item()])[0]
    item_info = df[df['POI_ID'] == item_id][['POI_ID', 'POI_NM', 'POI_TYPE']].iloc[0]
    # item_info에 'REVISIT_INTENTION', 'RCMDTN_INTENTION', 'STARS' 정보 추가
    item_info['REVISIT_INTENTION'] = data[data['item_id'] == item_id]['REVISIT_INTENTION'].iloc[0]
    item_info['RCMDTN_INTENTION'] = data[data['item_id'] == item_id]['RCMDTN_INTENTION'].iloc[0]
    item_info['STARS'] = data[data['item_id'] == item_id]['STARS'].iloc[0]
    recommended_items.append(item_info)

# 리스트를 데이터프레임으로 변환
recommended_items_df = pd.concat(recommended_items, axis=1).transpose()

# 결과 출력
print(f"사용자 {user_id}에 대한 Top-{top_k} 추천 아이템:")
display(recommended_items_df)

# FastAPI 코드

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import torch

app = FastAPI()

class UserIDRequest(BaseModel):
    user_id: str

@app.post("/recommend/")
async def recommend(request: UserIDRequest):
    user_id = request.user_id
    user_idx = user_label_encoder.transform([user_id])[0]

    lightGCN.eval()
    with torch.no_grad():
        # 사용자 임베딩 가져오기
        final_user_emb, final_item_emb, initial_user_emb, initial_item_emb = lightGCN.propagate_through_layers()
        user_embedding = final_user_emb[user_idx]

        # 아이템별 유사도 점수 계산
        similarity_scores = torch.matmul(user_embedding, final_item_emb.transpose(0, 1))
        recommended_item_indices = torch.topk(similarity_scores, 5).indices  # top_k를 5로 가정

    recommended_items = []

    for item_idx in recommended_item_indices:
        item_id = item_label_encoder.inverse_transform([item_idx.item()])[0]
        item_info = df[df['POI_ID'] == item_id][['POI_ID', 'POI_NM', 'POI_TYPE']].iloc[0].to_dict()
        item_info['REVISIT_INTENTION'] = data[data['item_id'] == item_id]['REVISIT_INTENTION'].iloc[0]
        item_info['RCMDTN_INTENTION'] = data[data['item_id'] == item_id]['RCMDTN_INTENTION'].iloc[0]
        item_info['STARS'] = data[data['item_id'] == item_id]['STARS'].iloc[0]
        recommended_items.append(item_info)

    return {"user_id": user_id, "recommendations": recommended_items}


In [ ]:
import pandas as pd

# '카멜리아'가 포함된 행을 필터링
sample = travel_log[travel_log['POI_NM'].str.contains('카멜리아') | travel_log['VISIT_AREA_NM'].str.contains('카멜리아')]

# 'TRAVELER_ID' 별로 'VISIT_AREA_TYPE_CD'의 빈도 계산
frequency = sample.groupby('TRAVELER_ID')['VISIT_AREA_TYPE_CD'].value_counts()

In [ ]:
frequency

In [ ]:
sample = travel_log[travel_log['POI_NM'].str.contains('카멜리아') | travel_log['VISIT_AREA_NM'].str.contains('카멜리아')]

In [ ]:
sample['TRAVELER_ID'].unique()

In [ ]:
# 모델 가중치 저장
torch.save(lightGCN.state_dict(), '../model/LightGCN_best_model_weights.pth')

In [ ]:
# 관광 메타 데이터 
df = pd.read_csv('../data/travel_meta/travel_meta.csv', encoding = 'utf-8-sig')